In [1]:
import os
new_path = 'yolov5'
os.chdir(new_path)
import platform
import sys

from yolov5.models.common import DetectMultiBackend
from yolov5.utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from yolov5.utils.general import (
    LOGGER,
    Profile,
    check_file,
    check_img_size,
    check_imshow,
    increment_path,
    non_max_suppression,
    scale_boxes,
)
from yolov5.utils.torch_utils import select_device, smart_inference_mode

In [2]:
from display_position_and_predict_moves import *
from corner_detection import *
from pieces_detection import *
from perspective_correction import *
from chess_position_utils import *
from convert_position_to_coordinates import *
from display_image_with_annotations import *

def compute_model_outputs_into_best_move(img, corner_boxes, pieces_boxes, stockfish_path, pieces_logo_directory_path, display_corners=True, display_boxes=False, display_coordinates=False, display_best_moves=True, display_evaluations=True, white_color=(255, 0, 0), black_color=(0, 0, 255)):

    corners = get_corner_coordinates_from_corner_boxes(corner_boxes)
    # Etape 3 : Calculer les coordinates des pièces à partir de leur box
    # (le milieu selon x, et le MEME milieu selon y)(il ne faut pas prendre le vrai milieu de la box car les pièces sont hautes mais pas larges)
    pieces_coordinates = []
    for box in pieces_boxes:
        delta = abs(box[2]-box[0])/2
        x = int(box[0] + delta)
        y = int(abs(box[3] - delta))
        # De la forme [x, y, proba, classe]
        pieces_coordinates.append([x, y, box[4], box[5]])

    # Etape 3 : Homographie
    output_size = 512   # Taille de l'image redressée (après transformation)
    # Redresser l'image et obtenir les nouvelles coordonnées des pièces
    image_redressee, new_pieces_coordinates, M = redresser_image(
        img.copy(), corners, pieces_coordinates, output_size)

    # Partie 4 : Choisir et tourner l'image dans le bon sens parmis les 4 possibilités (deux longues sous étapes)
    image_tournee, new_pieces_coordinates, center, angle = mettre_image_dans_le_bon_sens(
        image_redressee, new_pieces_coordinates, output_size)

    # Etape 5 : Déterminer les cases (h4...) des pièces sur l'échequier dans le bon sens
    pieces_positions = trouver_cases_pieces(
        new_pieces_coordinates, output_size)

    # Etape 6 : Conversion des cases en positions FEN pour être plus par Stockfish
    fen = coordinates_to_fen(pieces_positions)

    # Etape 6 : Visualisation d'un échequier virtuel
    ChessBoard = ChessPosition(fen, stockfish_path, pieces_logo_directory_path)
    # ChessBoard.plot_chess_board()

    # Etape 7 : Prédiction du meilleur coup par Stockfish
    best_move_white, best_move_black = ChessBoard.get_best_moves()
    white_bar_height, black_bar_height = ChessBoard.get_evaluation_bars()

    # Etape 8 : Convertir cette position h4 en coordonnées sur l'image de base (annuler tous les changements de perspectives et rotation de l'image effectuées)
    coordonnees_best_moves = convert_best_moves_to_original_image_coordinates(
        best_move_white, best_move_black, center, angle, M, output_size)

    # Etape 9 : Afficher les coordonnées de ces meilleurs coups sur l'image originale
    # image_annotated = place_best_move_on_image(cv2.imread(img_path), coordonnees_best_moves["white"], coordonnees_best_moves["black"])
    image_annotated = display_everything_on_original_image(img.copy(), corners, pieces_boxes, pieces_coordinates, coordonnees_best_moves, white_bar_height, black_bar_height,
                                                           display_corners=display_corners, display_boxes=display_boxes, display_coordinates=display_coordinates, display_best_moves=display_best_moves, display_evaluations=display_evaluations, white_color=white_color, black_color=black_color)

    return image_annotated

In [5]:
import os
%cd yolov5
%cd chess
current_directory = os.getcwd()
print("Répertoire actuel :", current_directory)

fichiers = os.listdir(current_directory)
print(fichiers)


[WinError 2] Le fichier spécifié est introuvable: 'yolov5'
c:\Users\tomma\OneDrive\Bureau\Mes_projets\Chess_detection\yolov5
c:\Users\tomma\OneDrive\Bureau\Mes_projets\Chess_detection\yolov5\chess
Répertoire actuel : c:\Users\tomma\OneDrive\Bureau\Mes_projets\Chess_detection\yolov5\chess
['chess_position_utils.py', 'convert_position_to_coordinates.py', 'corner_detection.py', 'display_image_with_annotations.py', 'display_position_and_predict_moves.py', 'model_functions.py', 'perspective_correction.py', 'pieces_detection.py']


In [13]:
from model_functions import * 

@smart_inference_mode()
def run(
    weights_corner_detection_model="yolov5s.pt",  # model path or triton URL
    weights_pieces_detection_model="yolov5s.pt",
    source= "data/images",  # file/dir/URL/glob/screen/0(webcam)
    stockfish_path = 'stockfish/stockfish-windows-x86-64.exe',
    pieces_logo_directory_path = 'chess_pieces',
    display_corners=True, 
    display_boxes=False, 
    display_coordinates=False, 
    display_best_moves=True, 
    display_evaluations=True, 
    white_color=(255, 0, 0), black_color=(0, 0, 255),
    data= "data/coco128.yaml",  # dataset.yaml path
    imgsz=(640, 640),  # inference size (height, width)
    conf_thres=0.25,  # confidence threshold
    iou_thres=0.45,  # NMS IOU threshold
    max_det=1000,  # maximum detections per image
    device="",  # cuda device, i.e. 0 or 0,1,2,3 or cpu
    view_img=True,  # show results
    save_txt=False,  # save results to *.txt
    save_crop=False,  # save cropped prediction boxes
    nosave=False,  # do not save images/videos
    classes=None,  # filter by class: --class 0, or --class 0 2 3
    agnostic_nms=False,  # class-agnostic NMS
    augment=False,  # augmented inference
    visualize=False,  # visualize features
    update=False,  # update all models
    project= "runs/detect",  # save results to project/name
    name="exp",  # save results to project/name
    exist_ok=False,  # existing project/name ok, do not increment
    line_thickness=3,  # bounding box thickness (pixels)
    hide_labels=False,  # hide labels
    hide_conf=False,  # hide confidences
    half=False,  # use FP16 half-precision inference
    dnn=False,  # use OpenCV DNN for ONNX inference
    vid_stride=1,  # video frame-rate stride
):
    
    source = str(source)
    save_img = not nosave and not source.endswith(".txt")  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(("rtsp://", "rtmp://", "http://", "https://"))
    webcam = source.isnumeric() or source.endswith(".streams") or (is_url and not is_file)
    screenshot = source.lower().startswith("screen")
    if is_url and is_file:
        source = check_file(source)  # download

    # Dossiers
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # créer un dossier incrémental
    (save_dir / "labels" if save_txt else save_dir).mkdir(parents=True, exist_ok=True)

    # CHARGEMENT DES DEUX MODELES
    device = select_device(device)
    model_corner = DetectMultiBackend(weights_corner_detection_model, device=device, dnn=dnn, data=data, fp16=half)
    model_pieces = DetectMultiBackend(weights_pieces_detection_model, device=device, dnn=dnn, data=data, fp16=half)  # second modèle
    stride, names, pt = model_corner.stride, model_corner.names, model_corner.pt
    imgsz = check_img_size(imgsz, s=stride)  # vérifier la taille de l'image

    # Création du dataloader
    bs = 1  # taille de batch
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Inférence
    model_corner.warmup(imgsz=(1 if pt or model_corner.triton else bs, 3, *imgsz))  # échauffement modèle 1
    model_pieces.warmup(imgsz=(1 if pt or model_pieces.triton else bs, 3, *imgsz))  # échauffement modèle 2
    seen, windows, dt = 0, [], (Profile(device=device), Profile(device=device), Profile(device=device))

    # Boucle sur le dataset
    for path, im, im0s, vid_cap, s in dataset:
        
        # Prétraitement
        with dt[0]:
            im, ims = preprocess_image(im, model_corner)  # même prétraitement pour les deux modèles
            
        # Inférence avec les deux modèles
        with dt[1]:
            pred1_corner = model_prediction(model_corner, im, ims, augment, visualize)
            pred2_pieces = model_prediction(model_pieces, im, ims, augment, visualize)
        
        # Application du NMS
        with dt[2]:
            pred1_corner = non_max_suppression(pred1_corner, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
            pred2_pieces = non_max_suppression(pred2_pieces, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        
        # Traitement des prédictions de chaque modèle
        for i, (corner_boxes, pieces_boxes) in enumerate(zip(pred1_corner, pred2_pieces)):  # pour chaque image
            seen += 1
            if webcam:  # si c'est une webcam
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, "frame", 0)

            # Chemins de sauvegarde
            p = Path(p)
            save_path = str(save_dir / p.name)
            
            # Traitement des prédictions du modèle 1
            if len(corner_boxes):
                corner_boxes[:, :4] = scale_boxes(im.shape[2:], corner_boxes[:, :4], im0.shape).round()
                if save_img or view_img:  # Ajout des boîtes de modèle 1
                    pass  # code pour dessiner les résultats de modèle 1
            
            # Traitement des prédictions du modèle 2
            if len(pieces_boxes):
                pieces_boxes[:, :4] = scale_boxes(im.shape[2:], pieces_boxes[:, :4], im0.shape).round()
                if save_img or view_img:  # Ajout des boîtes de modèle 2
                    pass  # code pour dessiner les résultats de modèle 2            
            corner_boxes = corner_boxes.numpy()
            pieces_boxes = pieces_boxes.numpy()
            
            # Etape 2 : Maintenant qu'on a les boxes, on utilise la grosse fonction
            try :
                image_annotated = compute_model_outputs_into_best_move(im0.copy(), corner_boxes, pieces_boxes, stockfish_path, pieces_logo_directory_path, 
                                                                       display_corners=display_corners, display_boxes=display_boxes, display_coordinates=display_coordinates, display_best_moves=display_best_moves, 
                                                                       display_evaluations=display_evaluations, white_color=white_color, black_color=black_color)
            except Exception:
                image_annotated = im0
                print('Problème de détection')
            # cv2.imshow("Image avec le déplacement", image_annotated)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
                
            # if view_img or save_img :
            #     pass       
            print(view_img)
            # Stream results
            if view_img:
                print("iiiin")
                if platform.system() == "Linux" and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), image_annotated.shape[1], image_annotated.shape[0])
                print('je suis dedans')
                cv2.imshow(str(p), image_annotated)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == "image":
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    save_video(i, save_path, vid_path, vid_writer, vid_cap, im0)


        # Print time (inference-only)
        LOGGER.info(f"{s}{'' if len(pieces_boxes) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")

    # Print results
    t = tuple(x.t / seen * 1e3 for x in dt)  # speeds per image
    LOGGER.info(f"Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}" % t)

In [14]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
    
#from yolov5.detect import run
corner_detection_model_path = 'c:/Users/tomma/OneDrive/Bureau/Mes_projets/Chess_detection/models/best_yolo_corner_detection.pt'
piece_detection_model_path = 'c:/Users/tomma/OneDrive/Bureau/Mes_projets/Chess_detection/models/best_yolov5x_pieces_detection.pt'
img_path = "c:/Users/tomma/OneDrive/Bureau/Mes_projets/Chess_detection/board_exemple_5.jpg"
stockfish_path = 'c:/Users/tomma/OneDrive/Bureau/Mes_projets/Chess_detection/stockfish/stockfish-windows-x86-64.exe'
pieces_logo_directory_path = 'c:/Users/tomma/OneDrive/Bureau/Mes_projets/Chess_detection/chess_pieces'
run(corner_detection_model_path, piece_detection_model_path, source=0, stockfish_path=stockfish_path, 
    pieces_logo_directory_path=pieces_logo_directory_path, display_corners=True, display_boxes=False, 
    display_coordinates=False, display_best_moves=True, display_evaluations=False, white_color=(255, 0, 0), 
    black_color=(0, 0, 255), view_img=True)

YOLOv5  v7.0-378-g2f74455a Python-3.10.5 torch-2.2.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Fusing layers... 
Model summary: 322 layers, 86254162 parameters, 0 gradients, 204.0 GFLOPs
WARNING  Environment does not support cv2.imshow() or PIL Image.show()

1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)



KeyboardInterrupt: 